In [1]:
#import SparkSession
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('random_forest').getOrCreate()

In [2]:
#read the dataset
df=spark.read.csv('affairs.csv',inferSchema=True,header=True)

### EDA

In [3]:
print((df.count(),len(df.columns)))

(6366, 6)


In [4]:
df.printSchema()

root
 |-- rate_marriage: integer (nullable = true)
 |-- age: double (nullable = true)
 |-- yrs_married: double (nullable = true)
 |-- children: double (nullable = true)
 |-- religious: integer (nullable = true)
 |-- affairs: integer (nullable = true)



In [7]:
df.describe().show(10,False)

+-------+------------------+------------------+-----------------+------------------+------------------+------------------+
|summary|rate_marriage     |age               |yrs_married      |children          |religious         |affairs           |
+-------+------------------+------------------+-----------------+------------------+------------------+------------------+
|count  |6366              |6366              |6366             |6366              |6366              |6366              |
|mean   |4.109644989004084 |29.082862079798932|9.00942507068803 |1.3968740182218033|2.4261702796104303|0.3224945020420987|
|stddev |0.9614295945655025|6.847881883668817 |7.280119972766412|1.433470828560344 |0.8783688402641785|0.467467779921086 |
|min    |1                 |17.5              |0.5              |0.0               |1                 |0                 |
|max    |5                 |42.0              |23.0             |5.5               |4                 |1                 |
+-------+-------

In [8]:
df.groupBy('rate_marriage','affairs').count().orderBy('rate_marriage','affairs','count',ascending=True).show()

+-------------+-------+-----+
|rate_marriage|affairs|count|
+-------------+-------+-----+
|            1|      0|   25|
|            1|      1|   74|
|            2|      0|  127|
|            2|      1|  221|
|            3|      0|  446|
|            3|      1|  547|
|            4|      0| 1518|
|            4|      1|  724|
|            5|      0| 2197|
|            5|      1|  487|
+-------------+-------+-----+



In [9]:
df.groupBy('religious','affairs').count().orderBy('religious','affairs','count',ascending=True).show()

+---------+-------+-----+
|religious|affairs|count|
+---------+-------+-----+
|        1|      0|  613|
|        1|      1|  408|
|        2|      0| 1448|
|        2|      1|  819|
|        3|      0| 1715|
|        3|      1|  707|
|        4|      0|  537|
|        4|      1|  119|
+---------+-------+-----+



In [10]:
df.groupBy('children','affairs').count().orderBy('children','affairs','count',ascending=True).show()

+--------+-------+-----+
|children|affairs|count|
+--------+-------+-----+
|     0.0|      0| 1912|
|     0.0|      1|  502|
|     1.0|      0|  747|
|     1.0|      1|  412|
|     2.0|      0|  873|
|     2.0|      1|  608|
|     3.0|      0|  460|
|     3.0|      1|  321|
|     4.0|      0|  197|
|     4.0|      1|  131|
|     5.5|      0|  124|
|     5.5|      1|   79|
+--------+-------+-----+



In [11]:
df.groupBy('affairs').mean().show()

+-------+------------------+------------------+------------------+------------------+------------------+------------+
|affairs|avg(rate_marriage)|          avg(age)|  avg(yrs_married)|     avg(children)|    avg(religious)|avg(affairs)|
+-------+------------------+------------------+------------------+------------------+------------------+------------+
|      1|3.6473453482708234|30.537018996590355|11.152459814905017|1.7289332683877252| 2.261568436434486|         1.0|
|      0| 4.329700904242986| 28.39067934152562| 7.989334569904939|1.2388128912589844|2.5045212149316023|         0.0|
+-------+------------------+------------------+------------------+------------------+------------------+------------+



### 特征工程

In [12]:
from pyspark.ml.feature import VectorAssembler

In [13]:
df_assembler = VectorAssembler(inputCols=['rate_marriage', 'age', 'yrs_married', 'children', 'religious'], outputCol="features")
df = df_assembler.transform(df)

In [16]:
df.printSchema()

root
 |-- rate_marriage: integer (nullable = true)
 |-- age: double (nullable = true)
 |-- yrs_married: double (nullable = true)
 |-- children: double (nullable = true)
 |-- religious: integer (nullable = true)
 |-- affairs: integer (nullable = true)
 |-- features: vector (nullable = true)



In [17]:
df.select(['features','affairs']).show(10,False)

+-----------------------+-------+
|features               |affairs|
+-----------------------+-------+
|[5.0,32.0,6.0,1.0,3.0] |0      |
|[4.0,22.0,2.5,0.0,2.0] |0      |
|[3.0,32.0,9.0,3.0,3.0] |1      |
|[3.0,27.0,13.0,3.0,1.0]|1      |
|[4.0,22.0,2.5,0.0,1.0] |1      |
|[4.0,37.0,16.5,4.0,3.0]|1      |
|[5.0,27.0,9.0,1.0,1.0] |1      |
|[4.0,27.0,9.0,0.0,2.0] |1      |
|[5.0,37.0,23.0,5.5,2.0]|1      |
|[5.0,37.0,23.0,5.5,2.0]|1      |
+-----------------------+-------+
only showing top 10 rows



### 构建数据集并划分

In [18]:
model_df=df.select(['features','affairs'])

In [19]:
train_df,test_df=model_df.randomSplit([0.75,0.25])

In [20]:
train_df.count()

4732

In [21]:
train_df.groupBy('affairs').count().show()

+-------+-----+
|affairs|count|
+-------+-----+
|      1| 1531|
|      0| 3201|
+-------+-----+



In [22]:
test_df.groupBy('affairs').count().show()

+-------+-----+
|affairs|count|
+-------+-----+
|      1|  522|
|      0| 1112|
+-------+-----+



### 构建和训练模型

In [23]:
from pyspark.ml.classification import RandomForestClassifier

In [24]:
rf_classifier=RandomForestClassifier(labelCol='affairs',numTrees=50).fit(train_df)

In [25]:
rf_predictions=rf_classifier.transform(test_df)

In [26]:
rf_predictions.show()

+--------------------+-------+--------------------+--------------------+----------+
|            features|affairs|       rawPrediction|         probability|prediction|
+--------------------+-------+--------------------+--------------------+----------+
|[1.0,17.5,0.5,0.0...|      0|[37.1702083767142...|[0.74340416753428...|       0.0|
|[1.0,22.0,2.5,1.0...|      1|[20.2973277809485...|[0.40594655561897...|       1.0|
|[1.0,22.0,2.5,1.0...|      1|[24.1537636858075...|[0.48307527371615...|       1.0|
|[1.0,27.0,2.5,0.0...|      1|[21.4813378494057...|[0.42962675698811...|       1.0|
|[1.0,27.0,6.0,1.0...|      0|[15.8462828560682...|[0.31692565712136...|       1.0|
|[1.0,27.0,6.0,3.0...|      0|[14.2467880500059...|[0.28493576100011...|       1.0|
|[1.0,27.0,9.0,1.0...|      1|[15.6749490875281...|[0.31349898175056...|       1.0|
|[1.0,27.0,9.0,4.0...|      0|[13.3351856217722...|[0.26670371243544...|       1.0|
|[1.0,27.0,13.0,2....|      1|[13.0525168470513...|[0.26105033694102...|    

### 预测

In [27]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator,MulticlassClassificationEvaluator

In [28]:
rf_accuracy=MulticlassClassificationEvaluator(labelCol='affairs',metricName='accuracy').evaluate(rf_predictions)

In [29]:
print('The accuracy of RF on test data is {0:.0%}'.format(rf_accuracy))

The accuracy of RF on test data is 72%


In [30]:
rf_precision=MulticlassClassificationEvaluator(labelCol='affairs',metricName='weightedPrecision').evaluate(rf_predictions)

In [31]:
print('The precision rate on test data is {0:.0%}'.format(rf_precision))

The precision rate on test data is 70%


In [32]:
rf_auc=BinaryClassificationEvaluator(labelCol='affairs').evaluate(rf_predictions)

In [33]:
print('The AUC on test data is {0:.0%}'.format(rf_auc))

The AUC on test data is 73%


### 变量贡献

In [34]:
rf_classifier.featureImportances

SparseVector(5, {0: 0.6258, 1: 0.018, 2: 0.2381, 3: 0.0524, 4: 0.0657})

In [35]:
df.schema["features"].metadata["ml_attr"]["attrs"]

{'numeric': [{'idx': 0, 'name': 'rate_marriage'},
  {'idx': 1, 'name': 'age'},
  {'idx': 2, 'name': 'yrs_married'},
  {'idx': 3, 'name': 'children'},
  {'idx': 4, 'name': 'religious'}]}

  ### 模型保存

In [38]:
rf_classifier.save("/Users/admin/Desktop/spark/RFmodel")

In [39]:
from pyspark.ml.classification import RandomForestClassificationModel

In [40]:
rf=RandomForestClassificationModel.load("/Users/admin/Desktop/spark/RFmodel")

In [41]:
model_preditions=rf.transform(test_df)

In [42]:
model_preditions.show()

+--------------------+-------+--------------------+--------------------+----------+
|            features|affairs|       rawPrediction|         probability|prediction|
+--------------------+-------+--------------------+--------------------+----------+
|[1.0,17.5,0.5,0.0...|      0|[37.1702083767142...|[0.74340416753428...|       0.0|
|[1.0,22.0,2.5,1.0...|      1|[20.2973277809485...|[0.40594655561897...|       1.0|
|[1.0,22.0,2.5,1.0...|      1|[24.1537636858075...|[0.48307527371615...|       1.0|
|[1.0,27.0,2.5,0.0...|      1|[21.4813378494057...|[0.42962675698811...|       1.0|
|[1.0,27.0,6.0,1.0...|      0|[15.8462828560682...|[0.31692565712136...|       1.0|
|[1.0,27.0,6.0,3.0...|      0|[14.2467880500059...|[0.28493576100011...|       1.0|
|[1.0,27.0,9.0,1.0...|      1|[15.6749490875281...|[0.31349898175056...|       1.0|
|[1.0,27.0,9.0,4.0...|      0|[13.3351856217722...|[0.26670371243544...|       1.0|
|[1.0,27.0,13.0,2....|      1|[13.0525168470513...|[0.26105033694102...|    